In [15]:
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer 
import nltk
from nltk.tokenize import word_tokenize 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [16]:
df_data=pd.DataFrame([{'Text': 'reading assignment', 'Label':'Ham'}, 
                      {'Text': 'analysis and design assignment', 'Label':'Ham'}, 
                      {'Text': 'hello how are you', 'Label':'Ham'}, 
                      {'Text': 'requirements analysis', 'Label':'Ham'}, 
                      {'Text': 'win free', 'Label':'Spam'}, 
                      {'Text': 'lotto millions', 'Label':'Spam'}, 
                      {'Text': 'win win win millions', 'Label':'Spam'},
                      {'Text': 'free offers', 'Label':'Spam'}])
df_data_ham=df_data[df_data.Label=='Ham']
df_data_spam=df_data[df_data.Label=='Spam']

In [17]:
#TDM(Term Document Frequency): all emails
vec=CountVectorizer()
tdf=vec.fit_transform(df_data['Text'])
total_words=len(vec.get_feature_names())

#TDM: ham
count_vec_ham=CountVectorizer()
tdf_ham=count_vec_ham.fit_transform(df_data_ham['Text'])
tdm_ham=pd.DataFrame(tdf_ham.toarray(), columns=count_vec_ham.get_feature_names())
word_list_ham=count_vec_ham.get_feature_names();
tdm_ham_dict=dict(zip(word_list_ham, tdf_ham.toarray().sum(axis=0)))
print(tdm_ham_dict)

#TDM: spam 
count_vect_spam = CountVectorizer() 
tdf_spam = count_vect_spam.fit_transform(df_data_spam['Text']) 
tdm_spam = pd.DataFrame(tdf_spam.toarray(), columns=count_vect_spam.get_feature_names()) 
word_list_spam = count_vect_spam.get_feature_names(); 
tdm_spam_dict = dict(zip(word_list_spam, tdf_spam.toarray().sum(axis=0) )) 
print(tdm_spam_dict)

{'analysis': 2, 'and': 1, 'are': 1, 'assignment': 2, 'design': 1, 'hello': 1, 'how': 1, 'reading': 1, 'requirements': 1, 'you': 1}
{'free': 2, 'lotto': 1, 'millions': 2, 'offers': 1, 'win': 4}


In [18]:
#compute probability for ham and spam
prior_prob_spam = df_data_spam.shape[0] / df_data.shape[0]
prior_prob_ham = df_data_ham.shape[0] / df_data.shape[0]

In [20]:
#calculate posterior probability 
def post_prob(test_sentence):

  prob_spam_all_words=1
  prob_ham_all_words=1

  ham_words_count=tdf_ham.toarray().sum()
  spam_words_count=tdf_spam.toarray().sum()

#P(word|spam)
  words=word_tokenize(test_sentence)
  for word in words:
    prob_spam_word=((tdm_spam_dict.get(word, 0) + 1) / (spam_words_count+total_words))
    prob_spam_all_words=prob_spam_all_words * prob_spam_word

    #P(word|ham)
    prob_ham_word = ((tdm_ham_dict.get(word, 0) + 1) / (ham_words_count + total_words))
    prob_ham_all_words = prob_ham_all_words * prob_ham_word

    # P(ham|words) = P(ham) * P(words|ham) 
    posterior_ham = prior_prob_ham * prob_ham_all_words 
    # P(spam|words) = P(spam) * P(words|spam) 
    posterior_spam = prior_prob_spam * prob_spam_all_words

    CLASS = 'ham' if posterior_ham > posterior_spam else 'spam'

    # prob 
    prob_ham = posterior_ham / (posterior_spam + posterior_ham) 
    prob_spam = posterior_spam / (posterior_spam + posterior_ham) 
    return (prob_ham, prob_spam, CLASS)

    prob_ham, prob_spam, c = posterior_prob(test_sentence='free deal') 
    print(prob_ham, prob_spam, c)